In [7]:
import sys
import random
from IPython.display import display
import torch
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import networkx as nx
from torch_geometric import utils
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from fastcore.all import *
from copy import deepcopy
import hdbscan
from torch_geometric.utils import degree
from environment_manager import *
from fastcore.all import *
import numpy as np
import os
import torch
from torch_geometric.data import Data, Dataset, DataLoader
import networkx as nx
from typing import List, Tuple
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from copy import deepcopy

env = load_create_environment('240318-initial parsing')
GHComponentTable.initialise()
component_idxes = GHComponentTable.df
guid_to_idx_dict = {id: i for i, id in enumerate(GHComponentTable.df['guid'])}
idx_to_guid_dict = {i: id for i, id in enumerate(GHComponentTable.df['guid'])}

# Define Transform Functions

In [15]:
def compute_and_assign_centralities(G, compute_degree=False, compute_closeness=False, compute_betweenness=False, compute_in_out_degree=False,compute_katz=False, compute_pagerank=False, compute_harmonic=False, compute_clustering=False, compute_coreness=False,  compute_hdbscan=False, hdbscan_min_cluster_size=5, **kwargs):
    if compute_degree:
        degree_centrality = nx.degree_centrality(G)
        nx.set_node_attributes(G, degree_centrality, 'degree_centrality')
    
    if compute_closeness:
        closeness_centrality = nx.closeness_centrality(G)
        nx.set_node_attributes(G, closeness_centrality, 'closeness_centrality')
    
    if compute_betweenness:
        betweenness_centrality = nx.betweenness_centrality(G)
        nx.set_node_attributes(G, betweenness_centrality, 'betweenness_centrality')

    
    if compute_katz:
        katz_centrality = nx.katz_centrality(G, **kwargs.get('katz', {}))
        nx.set_node_attributes(G, katz_centrality, 'katz_centrality')
    
    if compute_pagerank:
        pagerank = nx.pagerank(G, **kwargs.get('pagerank', {}))
        nx.set_node_attributes(G, pagerank, 'pagerank')
    
    if compute_harmonic:
        harmonic_centrality = nx.harmonic_centrality(G)
        nx.set_node_attributes(G, harmonic_centrality, 'harmonic_centrality')
    
    if compute_clustering:
        clustering_coefficient = nx.clustering(G)
        nx.set_node_attributes(G, clustering_coefficient, 'clustering_coefficient')
    
    if compute_coreness:
        # Directly assign the k-core number for each node without using max()
        coreness = nx.core_number(G)
        nx.set_node_attributes(G, coreness, 'coreness')

    
    if compute_in_out_degree:
        in_degree_centrality = nx.in_degree_centrality(G)
        out_degree_centrality = nx.out_degree_centrality(G)
        nx.set_node_attributes(G, in_degree_centrality, 'in_degree_centrality')
        nx.set_node_attributes(G, out_degree_centrality, 'out_degree_centrality')
        
    if compute_hdbscan:
           # Extract node positions
           positions = np.array([(G.nodes[node]['x'], G.nodes[node]['y']) for node in G.nodes()])
           
           # Apply HDBSCAN on spatial coordinates
           # Ensure that any DBSCAN-specific kwargs are not passed to HDBSCAN
           hdbscan_kwargs = {k: v for k, v in kwargs.items() if k.startswith('hdbscan_')}
           clustering = hdbscan.HDBSCAN(min_cluster_size=hdbscan_min_cluster_size, **hdbscan_kwargs).fit(positions)
           
           # Assign cluster labels to nodes
           for node, label in zip(G.nodes(), clustering.labels_):
               G.nodes[node]['hdbscan_cluster'] = label

def trnsfrm_positions(data):
    x_ = StandardScaler().fit_transform(np.array(data['x']).reshape(-1, 1))
    y_ = StandardScaler().fit_transform(np.array(data['y']).reshape(-1, 1))
    data['x'] = torch.tensor(np.concatenate([x_, y_], axis=1), dtype=torch.float32)

def trnsfm_compid(data, guid_to_idx_dict):
    assert data['x'] is not None, 'Please run trnsfrm_positions first'
    data['y'] = torch.tensor([guid_to_idx_dict[ids] for ids in data['compid']])
    ohe = OneHotEncoder(categories=[range(len(guid_to_idx_dict.values()))], handle_unknown='ignore')
    data['y'] = torch.tensor(ohe.fit_transform(data['y'].reshape(-1, 1)).toarray())
    return data

def trnsform_edges(data):
    if isinstance(data, np.ndarray):
        data = torch.tensor(data.toarray())
    t = torch.concat([torch.from_numpy(data[x]) for x in ['s_paramidx', 'r_paramidx', 's_access', 'r_access', 's_datamapping', 'r_datamapping']], dim=1)
    return t

def trnsfrm_paramidx(data):
    ohe = OneHotEncoder(categories=[range(11)], handle_unknown='ignore')
    p_in = ohe.fit_transform(np.array(data['s_paramidx'].reshape(-1, 1)))
    p_out = ohe.fit_transform(np.array(data['r_paramidx'].reshape(-1, 1)))
    data['s_paramidx'] = p_in.toarray()
    data['r_paramidx'] = p_out.toarray()

def trnsfrm_access(data):
    ohe = OneHotEncoder(categories=[range(3)], handle_unknown='ignore')
    s_access = ohe.fit_transform(np.array(data['s_access'].reshape(-1, 1)))
    r_access = ohe.fit_transform(np.array(data['r_access'].reshape(-1, 1)))
    data['s_access'] = s_access.toarray()
    data['r_access'] = r_access.toarray()

def trnsfrm_datamapping(data):
    ohe = OneHotEncoder(categories=[range(3)], handle_unknown='ignore')
    s_datamapping = ohe.fit_transform(np.array(data['s_datamapping'].reshape(-1, 1)))
    r_datamapping = ohe.fit_transform(np.array(data['r_datamapping'].reshape(-1, 1)))
    data['s_datamapping'] = s_datamapping.toarray()
    data['r_datamapping'] = r_datamapping.toarray()

def trsnfrm_dist(data):
    scaler = StandardScaler()
    data['distance'] = scaler.fit_transform(np.array(data['distance']).reshape(-1, 1))

def processX(data):
    std = {'x', 'edge_index', 'y', 'edge_features'}
    for feature in data.keys():  # Call keys() method to get an iterable view of the keys
        if feature in std:
            continue
        if data[feature].shape[0] == data['x'].shape[0]:
            # Apply scaler and reshape the feature to have the same dimension as 'x'
            if feature not in {'in_degree_centrality', 'out_degree_centrality'}:
                data[feature] = torch.tensor(StandardScaler().fit_transform(np.array(data[feature]).reshape(-1, 1)), dtype=torch.float32).view(-1)
            data['x'] = torch.cat([data['x'], data[feature].view(-1, 1)], dim=1)
            del data[feature]
def trnsfrm(data, guid_to_idx_dict, rmv_categories=None):
    if rmv_categories is None:
        rmv_categories = ['compid', 's_paramidx', 'r_paramidx', 's_access', 'r_access', 's_datamapping', 'r_datamapping', 'distance', 'r_is_optional', 's_is_optional', 'category']
    data_ = deepcopy(data)

    trnsfrm_positions(data_)
    trnsfm_compid(data_, guid_to_idx_dict)
    trnsfrm_paramidx(data_)
    trnsfrm_access(data_)
    trnsfrm_datamapping(data_)
    trsnfrm_dist(data_)
    data_['edge_features'] = trnsform_edges(data_)
    for ftr in rmv_categories:
        del data_[ftr]

    processX(data_)

    return data_

In [16]:


class GraphMLDataset(Dataset):
    def __init__(self, root_dir: str, graphml_files: List[str] = None, num_files: int = None, transform=None, pre_transform=None):
        super().__init__(root_dir, transform, pre_transform)
        if graphml_files is None:
            self.graphml_files = [file for file in os.listdir(root_dir) if file.endswith(".graphml")]
            if num_files is not None:
                self.graphml_files = random.sample(self.graphml_files, num_files)
        else:
            self.graphml_files = graphml_files

    @property
    def raw_file_names(self) -> List[str]:
        return self.graphml_files

    def len(self) -> int:
        return len(self.graphml_files)

    def get(self, idx: int) -> Data:
        graphml_file = self.graphml_files[idx]
        graphml_path = os.path.join(self.root, graphml_file)
        graph = nx.read_graphml(graphml_path)

        # Remove isolated nodes
        graph.remove_nodes_from(list(nx.isolates(graph)))

        # Compute and assign centralities
        compute_and_assign_centralities(graph, compute_betweenness=True, compute_in_out_degree=True)

        data = utils.from_networkx(graph)
        return data


def get_dataloader(root_dir: str, guid_to_idx_dict, batch_size: int, graphml_files: List[str] = None, num_files: int = None, shuffle: bool = True) -> Tuple[DataLoader, DataLoader]:
    dataset = GraphMLDataset(root_dir, graphml_files=graphml_files, num_files=num_files, transform=lambda data: trnsfrm(data, guid_to_idx_dict))
    num_nodes = sum(data.num_nodes for data in dataset)
    perm = torch.randperm(num_nodes)
    train_size = int(num_nodes * 0.7)
    val_size = int(num_nodes * 0.2)
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = deepcopy(train_mask)
    test_mask = deepcopy(train_mask)

    train_mask[perm[:train_size]] = True
    val_mask[perm[train_size:train_size+val_size]] = True
    test_mask[perm[train_size+val_size:]] = True

    train_dataset = dataset[train_mask]
    val_dataset = dataset[val_mask]
    test_dataset = dataset[test_mask]

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

    return train_loader, val_loader, test_loader

# # Usage example
# root_directory = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\ExtractionEnvironments\240318-initial parsing\05-GraphML\01599_Sketch ModelsJS.graphml"
# guid_to_idx_dict = {id: i for i, id in enumerate(GHComponentTable.df['guid'])}
# batch_size = 32
# 
# train_loader, val_loader, test_loader = get_dataloader(root_directory, guid_to_idx_dict, batch_size)


In [17]:
# Usage examples
root_directory = env.dirs['graphml']
guid_to_idx_dict = {id: i for i, id in enumerate(GHComponentTable.df['guid'])}
batch_size = 32

# Load a single GraphML file
single_file = "01599_Sketch ModelsJS.graphml"
train_loader, val_loader, test_loader = get_dataloader(root_directory, guid_to_idx_dict, batch_size, graphml_files=[single_file])

# # Load all GraphML files from a folder
# train_loader, val_loader, test_loader = get_dataloader(root_directory, guid_to_idx_dict, batch_size)
# 
# # Load a random subset of GraphML files from a folder
# num_files = 10
# train_loader, val_loader, test_loader = get_dataloader(root_directory, guid_to_idx_dict, batch_size, num_files=num_files)

IndexError: range object index out of range